In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("what-makes-a-great-software-engineer.pdf")
data = loader.load()
# data

In [ ]:
len(data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500)
docs = text_splitter.split_documents(data)

print(type(docs))

In [ ]:
import os
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.environ['GOOGLE_API_KEY'])
vector = embeddings.embed_query("hello, world!")
vector[:5]


In [7]:
# import chromadb.utils.embedding_functions as embedding_functions
# google_embed_func = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
# google_embed_func(docs)
# GoogleGenerativeAIEmbeddings
load_dotenv()
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k": 10})
retrieved_docs = retriever.invoke("What are some skills to be a good software engineer?")
len(retrieved_docs)


In [ ]:
for retrieved_doc in retrieved_docs:
    # print(retrieved_doc.page_content)

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temprature=0, max_tokens=300)


In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for answering questions. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know."
    "Keep your answer precise and format your answer in bullet points."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [16]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What are some skills to be a good software engineer?"})
print(response["answer"])

Based on the provided text, here are some attributes of great software engineers:

*   **Ability to learn on the job:** Continuously improving and learning new skills is critical.
*   **Creating elegant software/designs:**  Producing simple, intuitive, and easily understandable software.
*   **Human judgment, inspiration, and creativity:** These are considered more important than just technical knowledge.

The text also mentions that most attributes focus on *how* to achieve software engineering goals, rather than just *what* skills and knowledge are needed.  Therefore, while specific technical skills are implied, the emphasis is on the application and development of those skills.